In [4]:
!git version

git version 2.28.0.windows.1


In [6]:
!git init

Initialized empty Git repository in C:/Users/Emre/Downloads/web_scraping_workshop/.git/


In [3]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

# import chrome driver manager
from webdriver_manager.chrome import ChromeDriverManager

# install the latest version of chrome driver
driver = webdriver.Chrome(ChromeDriverManager().install())




====== WebDriver manager ======
Could not get version for google-chrome with the command:  powershell "$ErrorActionPreference='silentlycontinue' ; (Get-Item -Path "$env:PROGRAMFILES\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion ; if (-not $? -or $? -match $error) { (Get-Item -Path "$env:PROGRAMFILES(x86)\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion } if (-not $? -or $? -match $error) { (Get-Item -Path "$env:LOCALAPPDATA\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion } if (-not $? -or $? -match $error) { reg query "HKCU\SOFTWARE\Google\Chrome\BLBeacon" /v version } if (-not $? -or $? -match $error) { reg query "HKLM\SOFTWARE\Wow6432Node\Microsoft\Windows\CurrentVersion\Uninstall\Google Chrome" /v version }"
Current google-chrome version is UNKNOWN
Get LATEST chromedriver version for UNKNOWN google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/98.0.4758.102/chromedriver_win32.zip
Driver has been 

In [1]:
pwd

'C:\\Users\\Emre\\Downloads\\web_scraping_workshop'

In [1]:
#driver = webdriver.Chrome()
#driver.get("https://www.ajc.com/senior-care-quality-report/homes/")

In [5]:
soup = BeautifulSoup(driver.page_source, "lxml")

In [6]:
# there are 398 facilities
len(soup.find(class_="table b-table").find_all("a"))

398

In [7]:
# Get list of facility links
facility_link_list = driver.find_elements_by_css_selector('.list-item-name')
len(facility_link_list)

398

In [9]:
# check outer HTML for the first link
facility_link_list[0].get_attribute('outerHTML')

'<a data-v-052e3680="" href="/senior-care-quality-report/homes/addington-place-at-the-mills/" class="list-item-name">\n        Addington Place at the Mills\n      </a>'

In [10]:
# check facility name for the first link
facility_link_list[0].get_attribute('innerHTML').strip()

'Addington Place at the Mills'

In [12]:
#below code clicks 1st facility (Addington Place at the Mills) link:
facility_link_list[0].click()
#get the current url for facility page
driver.current_url

'https://www.ajc.com/senior-care-quality-report/homes/'

In [18]:
# find detail buttons on the facility page and click each:
details_buttons = driver.find_elements_by_xpath("//button[@data-v-ee0ef674=''][@type='button']")
#<input name="continue" type="button" value="Clear" />
#'//div[@id="abc"]//span[@class="btn"]'
for button in details_buttons:
    button.click()
    
# save facility detailed page after clicking buttons above:
facilitypage_detailed = BeautifulSoup(driver.page_source, "lxml")

# save this as an html file:
html_file = open(file = 'facility_pages/'+facilitypage_detailed.find(class_='h3').text.strip()+'.html', mode = "w", encoding = "utf-8")
html_file.write(facilitypage_detailed.text)
html_file.close()

OSError: [Errno 22] Invalid argument: 'https://www.ajc.com/senior-care-quality-report/homes/addington-place-at-the-mills/.html'

In [27]:
#import pandas to create data frames
import pandas as pd

In [35]:
#example empty data frame
data = []

new_row = {'name':'Geo', 'physics':87, 'chemistry':92, 'algebra':97}

data.append(new_row)
pd.DataFrame(data)

,name,physics,chemistry,algebra
0,Geo,87,92,97


In [38]:
name = facilitypage_detailed.find(class_='h3').text.strip()
name

'Addington Place at the Mills'

In [66]:
address = facilitypage_detailed.find(class_='pt-2').text.strip()
address

'550 Barrington Dr, Roswell, GA 30075'

In [67]:
factype = facilitypage_detailed.find(class_='text-muted').text.strip()
factype

'Personal Care Home'

In [68]:
facilitypage_detailed.find(class_='pb-md-4').text.strip().split()

['License:', 'PCH008668', 'Beds:', '75', 'website']

In [69]:
license = facilitypage_detailed.find(class_='pb-md-4').text.strip().split()[1]
license

'PCH008668'

In [70]:
beds = facilitypage_detailed.find(class_='pb-md-4').text.strip().split()[3]
beds

'75'

In [71]:
website = facilitypage_detailed.find(class_='pb-md-4').find(name = 'a')['href']
website

'https://www.cedarhurstliving.com/location/addington-place-the-mills/'

In [72]:
#Get Violations per routine inspection for this facility
avg_viol = facilitypage_detailed.find_all(class_='annotation')[1].text
avg_viol

'8.4'

In [73]:
#facilities main info data frame
facilities_data = []

new_row = {'Name':name, 'Address':address, 'Type':factype, 'License':license, 'Beds':beds, 
           'Website':website, 'Avg. Violations':avg_viol}

facilities_data.append(new_row)
pd.DataFrame(facilities_data)

,Name,Address,Type,License,Beds,Website,Avg. Violations
0,Addington Place at the Mills,"550 Barrington Dr, Roswell, GA 30075",Personal Care Home,PCH008668,75,https://www.cedarhurstliving.com/location/addi...,8.4


## Filling out inspection records informaiton:

In [80]:
inspections_data = []

for inspection in facilitypage_detailed.find_all(class_='inspections-tr'):
    sev2 = 0; sev1 =0; harm=0; prev1=0; prev2=0; prev3=0;
    
    if  'b-table-has-details' in inspection['class']:
        insp_date = inspection.find_all('td')[0].text.strip()
        insp_type = inspection.find_all('td')[1].text.strip()
        num_viol = inspection.find_all('td')[2].text.strip()
        inspection_row = {'date':insp_date, 'name':name, 'license':license, 'type':insp_type, 'Num of Volations':num_viol, 
                      'High severe':sev2, 'Medium severe':sev1, 'Harm':harm, 'Prev1':prev1, 'Prev2':prev2, 'Prev3':prev3}
        #print("On the date:",insp_date," type:",insp_type,num_viol,"violations recorded.")
    else:
        for hitbox in inspection(class_='fa-hitbox'):
            sev2+=1 if 'severity2' in hitbox['class'] else 0
            sev1+=1 if 'severity1' in hitbox['class'] else 0
            prev1+=1 if 'prev1' in hitbox['class'] else 0
            prev2+=1 if 'prev2' in hitbox['class'] else 0
            prev3+=1 if 'prev3' in hitbox['class'] else 0
            harm+=1 if 'harm1' in hitbox['class'] else 0
    #inspection_row('High severe') = sev2
        inspection_row = {'date':insp_date, 'name':name, 'license':license, 'type':insp_type, 'Num of Volations':num_viol, 
                      'High severe':sev2, 'Medium severe':sev1, 'Harm':harm, 'Prev1':prev1, 'Prev2':prev2, 'Prev3':prev3}
        inspections_data.append(inspection_row)

#data frame:
pd.DataFrame(inspections_data)

,date,name,license,type,Num of Volations,High severe,Medium severe,Harm,Prev1,Prev2,Prev3
0,12/9/19,Addington Place at the Mills,PCH008668,routine,-,0,0,0,0,0,0
1,6/20/19,Addington Place at the Mills,PCH008668,complaint,1,1,0,1,0,0,0
2,6/3/19,Addington Place at the Mills,PCH008668,complaint,2,1,1,1,0,0,0
3,4/12/19,Addington Place at the Mills,PCH008668,Routine / Complaint,6,2,3,0,1,0,0
4,1/12/18,Addington Place at the Mills,PCH008668,complaint,1,0,1,0,0,0,0
5,12/19/17,Addington Place at the Mills,PCH008668,other,-,0,0,0,0,0,0
6,12/18/17,Addington Place at the Mills,PCH008668,complaint,-,0,0,0,0,0,0
7,12/7/17,Addington Place at the Mills,PCH008668,complaint,-,0,0,0,0,0,0
8,9/18/17,Addington Place at the Mills,PCH008668,complaint,4,1,1,0,0,0,0
9,9/8/17,Addington Place at the Mills,PCH008668,complaint,-,0,0,0,0,0,0


In [91]:
#import webdriver wait:
from selenium.webdriver.support.ui import WebDriverWait

In [124]:
#Go back to the previous (main) page:
driver.back()
#if that fails use this: driver.execute_script("window.history.go(-1)")
#or forward: driver.execute_script("window.history.go(+1)")

In [93]:
# Get list of facility links again
facility_link_list = driver.find_elements_by_css_selector('.list-item-name')

#below code clicks 2nd facility (Addington Place of Alpharetta) link:
facility_link_list[1].click()

#wait 3 seconds for page to load
WebDriverWait(driver, 3)

#get the current url for facility page
driver.current_url

'https://www.ajc.com/senior-care-quality-report/homes/addington-place-of-alpharetta/'

## Now we iterate over 398 facilities links to get their informaiton

In [271]:
# prepare silent run
from selenium.webdriver.chrome.options import Options

options = Options()
options.headless = True

# install the latest version of chrome driver
driver = webdriver.Chrome(options=options, executable_path=r'C:\Users\Emre\Downloads\web_scraping_workshop\chromedriver.exe')

In [269]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

options = Options()
options.headless = True
driver = webdriver.Chrome(options=options, executable_path=r'C:\Users\Emre\Downloads\web_scraping_workshop\chromedriver.exe')
driver.get("http://google.com/")
print ("Headless Chrome Initialized")
driver.quit()

Headless Chrome Initialized


In [176]:
# use regular chrome driver
# import chrome driver manager
from webdriver_manager.chrome import ChromeDriverManager

# install the latest version of chrome driver
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Could not get version for google-chrome with the command:  powershell "$ErrorActionPreference='silentlycontinue' ; (Get-Item -Path "$env:PROGRAMFILES\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion ; if (-not $? -or $? -match $error) { (Get-Item -Path "$env:PROGRAMFILES(x86)\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion } if (-not $? -or $? -match $error) { (Get-Item -Path "$env:LOCALAPPDATA\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion } if (-not $? -or $? -match $error) { reg query "HKCU\SOFTWARE\Google\Chrome\BLBeacon" /v version } if (-not $? -or $? -match $error) { reg query "HKLM\SOFTWARE\Wow6432Node\Microsoft\Windows\CurrentVersion\Uninstall\Google Chrome" /v version }"
Current google-chrome version is UNKNOWN
Get LATEST chromedriver version for UNKNOWN google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/98.0.4758.102/chromedriver_win32.zip
Driver has been 

In [186]:
# lets separate 398 facilities into chunks of 10:
# special thanks to stackoverflow: https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
myfaclist = range(398) 

def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
        
for chunk in chunks(myfaclist, 10):
    #print(chunk)
    print(list(chunk))
    #for el in chunk:
    #    print(el)
    #break

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
[20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
[30, 31, 32, 33, 34, 35, 36, 37, 38, 39]
[40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
[50, 51, 52, 53, 54, 55, 56, 57, 58, 59]
[60, 61, 62, 63, 64, 65, 66, 67, 68, 69]
[70, 71, 72, 73, 74, 75, 76, 77, 78, 79]
[80, 81, 82, 83, 84, 85, 86, 87, 88, 89]
[90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
[100, 101, 102, 103, 104, 105, 106, 107, 108, 109]
[110, 111, 112, 113, 114, 115, 116, 117, 118, 119]
[120, 121, 122, 123, 124, 125, 126, 127, 128, 129]
[130, 131, 132, 133, 134, 135, 136, 137, 138, 139]
[140, 141, 142, 143, 144, 145, 146, 147, 148, 149]
[150, 151, 152, 153, 154, 155, 156, 157, 158, 159]
[160, 161, 162, 163, 164, 165, 166, 167, 168, 169]
[170, 171, 172, 173, 174, 175, 176, 177, 178, 179]
[180, 181, 182, 183, 184, 185, 186, 187, 188, 189]
[190, 191, 192, 193, 194, 195, 196, 197, 198, 199]
[200, 201, 202, 203, 204, 205, 206, 207, 208, 209]
[210, 211, 212, 213, 214, 215, 216, 217

In [294]:
'Harbor at Renaissance Marquis, The / Renaissance Marquis Retirement Village'.partition("/")[0].strip()

'Harbor at Renaissance Marquis, The'

In [292]:
facilitypage_detailed = BeautifulSoup(driver.page_source, "lxml")
# some facility names are wierd
facilitypage_detailed.find(class_='h3').text.strip() #'Cottage Landing Memory Care Building / Cottage Landing'
facilitypage_detailed.find(class_='h3').text.strip().partition("/")[0].strip()
name.partition("/")[0].strip()

'Addington Place of Alpharetta'

In [247]:
# Facility 37 does not have a website, so the code broke!
# save facility detailed page in current page:
facilitypage_detailed = BeautifulSoup(driver.page_source, "lxml")
# need to introduce try in website  command:

try: website = facilitypage_detailed.find(class_='pb-md-4').find(name = 'a')['href']
except Exception: 
    website = ""

website

''

In [296]:
# import tqdm for progress bar:
#from tqdm import tqdm

# A function that scraps n facilities each time from a given interval

def myscrapper (mylist):

    import time

    #navigate to main AJC website
    driver.get("https://www.ajc.com/senior-care-quality-report/homes/")

    #facilities main info data frame
    facilities_data = []
    inspections_data = []

    #for i, x in enumerate(tqdm(range(10))):

    for i in mylist:
        
        print(i)

        #wait 3 seconds for page to load
        WebDriverWait(driver, 15)
        
        time.sleep(1) #sleep for 1 sec

        #below code clicks ith facility link:
        driver.find_element_by_css_selector(" thead > tr > th:nth-child(2)").location_once_scrolled_into_view

        facility_link_list = driver.find_elements_by_css_selector('.list-item-name')

        if i > 3:
            facility_link_list[i-1].location_once_scrolled_into_view

        # Get list of facility links
        facility_link_list = driver.find_elements_by_css_selector('.list-item-name')

        facility_link_list[i].click()

        #wait 3 seconds for page to load
        WebDriverWait(driver, 15)

        #get the current url for facility page
        #driver.current_url

        #element = WebDriverWait(driver, 2).until(
        #    EC.presence_of_element_located((By.PARTIAL_LINK_TEXT, "Followers"))
        #)

        time.sleep(2) #sleep for 1 sec

        mylocator = driver.find_element_by_css_selector("ul > li:nth-child(6) > strong")
        mylocator.location_once_scrolled_into_view

        time.sleep(1) #sleep for 1 sec

        # find detail buttons on the facility page and click each:
        details_buttons = driver.find_elements_by_css_selector("tbody > tr   button")
        #details_buttons = driver.find_elements_by_xpath("//button[@data-v-ee0ef674=''][@type='button']")
        #details_buttons[0].click()

        time.sleep(1) #sleep for 1 sec

        for button in details_buttons:
            #print(button)
            #time.sleep(1) #sleep for 1 sec
            #button.location_once_scrolled_into_view
            button.click()


        # save facility detailed page after clicking buttons above:
        facilitypage_detailed = BeautifulSoup(driver.page_source, "lxml")

        # get facility name
        name = facilitypage_detailed.find(class_='h3').text.strip()

        # save this as an html file:
        html_file = open(file = 'facility_pages/'+name.partition("/")[0].strip()+'.html', mode = "w", encoding = "utf-8")
        html_file.write(facilitypage_detailed.text)
        html_file.close()

        # main facility info
        address = facilitypage_detailed.find(class_='pt-2').text.strip()
        factype = facilitypage_detailed.find(class_='text-muted').text.strip()
        license = facilitypage_detailed.find(class_='pb-md-4').text.strip().split()[1]
        beds = facilitypage_detailed.find(class_='pb-md-4').text.strip().split()[3]
        
        #some facilities don't have website info, so we try :)
        try: website = facilitypage_detailed.find(class_='pb-md-4').find(name = 'a')['href']
        except Exception: 
            website = ""
        
        #Get Violations per routine inspection for this facility
        facilitypage_detailed.find_all(class_='annotation')[1].location_once_scrolled_into_view
        avg_viol = facilitypage_detailed.find_all(class_='annotation')[1].text


        new_row = {'Name':name, 'Address':address, 'Type':factype, 'License':license, 'Beds':beds, 
               'Website':website, 'Avg. Violations':avg_viol}
        facilities_data.append(new_row)


        for inspection in facilitypage_detailed.find_all(class_='inspections-tr'):
            sev2 = 0; sev1 =0; harm=0; prev1=0; prev2=0; prev3=0;
            if  'b-table-has-details' in inspection['class']:
                insp_date = inspection.find_all('td')[0].text.strip()
                insp_type = inspection.find_all('td')[1].text.strip()
                num_viol = inspection.find_all('td')[2].text.strip()
                inspection_row = {'date':insp_date, 'name':name, 'license':license, 'type':insp_type, 'Num of Volations':num_viol, 
                              'High severe':sev2, 'Medium severe':sev1, 'Harm':harm, 'Prev1':prev1, 'Prev2':prev2, 'Prev3':prev3}
                #print("On the date:",insp_date," type:",insp_type,num_viol,"violations recorded.")
            elif 'b-table-details' in inspection['class']:
                for hitbox in inspection(class_='fa-hitbox'):
                    sev2+=1 if 'severity2' in hitbox['class'] else 0
                    sev1+=1 if 'severity1' in hitbox['class'] else 0
                    prev1+=1 if 'prev1' in hitbox['class'] else 0
                    prev2+=1 if 'prev2' in hitbox['class'] else 0
                    prev3+=1 if 'prev3' in hitbox['class'] else 0
                    harm+=1 if 'harm1' in hitbox['class'] else 0
            #inspection_row('High severe') = sev2
                inspection_row = {'date':insp_date, 'name':name, 'license':license, 'type':insp_type, 'Num of Volations':num_viol, 
                              'High severe':sev2, 'Medium severe':sev1, 'Harm':harm, 'Prev1':prev1, 'Prev2':prev2, 'Prev3':prev3}
                inspections_data.append(inspection_row)
            else: None


        print("Facility ",i+1," th is done!")
        #Go back to the previous (main) page:
        driver.back()

    #data frame:
    #pd.DataFrame(facilities_data)
    #pd.DataFrame(inspections_data)
    return facilities_data, inspections_data

### Use myscrapper function on the chunks of the facilities list

In [212]:
count = 0
for chunk in chunks(myfaclist, 10):
    #print(chunk)
    count+=1

print(count) # I have 40 chunks
print(len(list(chunks(myfaclist, 10)))) #confirm

40
40


In [214]:
# NOTE: THE FOLLOWING IS NOT BEST PRACTICE BUT IT WORKS FOR NOW!
# here is a way to create dynamic variable names
for x in range(0, 4):
    globals()['string%s' % (x+1)] = 'Hello'

print(string1, string2, string3, string4)

Hello Hello Hello Hello


In [217]:
globals()['string%s' % (1)], globals()['string%s' % (2)] = "gel", "git"
print(string1, string2)

gel git


In [238]:
chunk_num = 0

for chunk in chunks(myfaclist, 10):
    chunk_num+=1
    #print(chunk)
    if chunk_num in [1,2,3]:
        continue
        
    print(list(chunk))


[30, 31, 32, 33, 34, 35, 36, 37, 38, 39]
[40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
[50, 51, 52, 53, 54, 55, 56, 57, 58, 59]
[60, 61, 62, 63, 64, 65, 66, 67, 68, 69]
[70, 71, 72, 73, 74, 75, 76, 77, 78, 79]
[80, 81, 82, 83, 84, 85, 86, 87, 88, 89]
[90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
[100, 101, 102, 103, 104, 105, 106, 107, 108, 109]
[110, 111, 112, 113, 114, 115, 116, 117, 118, 119]
[120, 121, 122, 123, 124, 125, 126, 127, 128, 129]
[130, 131, 132, 133, 134, 135, 136, 137, 138, 139]
[140, 141, 142, 143, 144, 145, 146, 147, 148, 149]
[150, 151, 152, 153, 154, 155, 156, 157, 158, 159]
[160, 161, 162, 163, 164, 165, 166, 167, 168, 169]
[170, 171, 172, 173, 174, 175, 176, 177, 178, 179]
[180, 181, 182, 183, 184, 185, 186, 187, 188, 189]
[190, 191, 192, 193, 194, 195, 196, 197, 198, 199]
[200, 201, 202, 203, 204, 205, 206, 207, 208, 209]
[210, 211, 212, 213, 214, 215, 216, 217, 218, 219]
[220, 221, 222, 223, 224, 225, 226, 227, 228, 229]
[230, 231, 232, 233, 234, 235, 236, 237, 238, 239]

In [297]:
import time # I want to time my code
start_time = time.time()

myfaclist = range(398) 

def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

chunk_num = 0

for chunk in chunks(myfaclist, 10):
    chunk_num+=1
    #print(chunk)
    if chunk_num in [1,2,3,4,5,6,7,8,9,10]:
        continue
        
    print(list(chunk))
    globals()['facilities_data_chunk%s' % (chunk_num)], globals()['inspections_data_chunk%s' % (chunk_num)] = myscrapper(chunk)
    #if chunk_num ==3:
    #    break
    
# How long did it take
print("--- %s seconds ---" % (time.time() - start_time))

[100, 101, 102, 103, 104, 105, 106, 107, 108, 109]
100
Facility  101  th is done!
101
Facility  102  th is done!
102
Facility  103  th is done!
103
Facility  104  th is done!
104
Facility  105  th is done!
105
Facility  106  th is done!
106
Facility  107  th is done!
107
Facility  108  th is done!
108
Facility  109  th is done!
109
Facility  110  th is done!
[110, 111, 112, 113, 114, 115, 116, 117, 118, 119]
110
Facility  111  th is done!
111
Facility  112  th is done!
112
Facility  113  th is done!
113
Facility  114  th is done!
114
Facility  115  th is done!
115
Facility  116  th is done!
116
Facility  117  th is done!
117
Facility  118  th is done!
118
Facility  119  th is done!
119
Facility  120  th is done!
[120, 121, 122, 123, 124, 125, 126, 127, 128, 129]
120
Facility  121  th is done!
121
Facility  122  th is done!
122
Facility  123  th is done!
123
Facility  124  th is done!
124
Facility  125  th is done!
125
Facility  126  th is done!
126
Facility  127  th is done!
127
Facili

Facility  328  th is done!
328
Facility  329  th is done!
329
Facility  330  th is done!
[330, 331, 332, 333, 334, 335, 336, 337, 338, 339]
330
Facility  331  th is done!
331
Facility  332  th is done!
332
Facility  333  th is done!
333
Facility  334  th is done!
334
Facility  335  th is done!
335
Facility  336  th is done!
336
Facility  337  th is done!
337
Facility  338  th is done!
338
Facility  339  th is done!
339
Facility  340  th is done!
[340, 341, 342, 343, 344, 345, 346, 347, 348, 349]
340
Facility  341  th is done!
341
Facility  342  th is done!
342
Facility  343  th is done!
343
Facility  344  th is done!
344
Facility  345  th is done!
345
Facility  346  th is done!
346
Facility  347  th is done!
347
Facility  348  th is done!
348
Facility  349  th is done!
349
Facility  350  th is done!
[350, 351, 352, 353, 354, 355, 356, 357, 358, 359]
350
Facility  351  th is done!
351
Facility  352  th is done!
352
Facility  353  th is done!
353
Facility  354  th is done!
354
Facility  

In [298]:
pd.DataFrame(facilities_data_chunk10)

,Name,Address,Type,License,Beds,Website,Avg. Violations
0,Cadence Marietta,"1345 Milford Church Rd SW, Marietta, GA 30008",Assisted Living Community,ALC000560,133,https://cadencemarietta.com,3.3
1,Camellia Place,"294 Rope Mill Rd, Woodstock, GA 30188",Assisted Living Community,ALC000210,110,https://www.camelliaplace.com,1.0
2,Camellia Walk Assisted Living and Memory Care,"3949 Evans to Locks Rd, Evans, GA 30809",Assisted Living Community,ALC000254,95,https://camelliawalk.com,1.2
3,Cameron Hall of Ellijay,"114 Penland St, Ellijay, GA 30540",Personal Care Home,PCH008333,70,https://www.fivestarseniorliving.com/communiti...,4.0
4,Cameron Hall Senior Living Communities,"240 Marietta Hwy, Canton, GA 30114",Assisted Living Community,ALC000107,105,https://www.fivestarseniorliving.com/communiti...,3.3
5,Cannon Wood Village,"2834 Old 441 S, Tiger, GA 30576",Personal Care Home,119020051,32,https://www.cannonwoodvillage.com,10.7
6,Canterfield of Kennesaw,"4381 Bells Ferry Rd, Kennesaw, GA 30144",Assisted Living Community,ALC000252,99,http://www.canterfieldofkennesaw.com,1.5
7,Carlyle Place Retirement Commu,"5300 Zebulon Rd, Macon, GA 31210",Personal Care Home,11030141,66,tel:478-405-4500,1.5
8,Carriage House of Southern Plantation,"690 Tommy Lee Fuller Dr, Loganville, GA 30052",Personal Care Home,PCH006952,43,https://www.seniorcaring.com/care-in-loganvill...,9.5
9,Cedar Hill Senior Living Community,"402 E Ellawood Ave, Cedartown, GA 30125",Personal Care Home,PCH008895,60,https://www.charterseniorliving.com/our-commun...,3.4


In [276]:
pd.DataFrame(inspections_data_chunk10)

,date,name,license,type,Num of Volations,High severe,Medium severe,Harm,Prev1,Prev2,Prev3
0,10/21/19,Cadence Marietta,ALC000560,complaint,-,0,0,0,0,0,0
1,9/19/19,Cadence Marietta,ALC000560,complaint,-,0,0,0,0,0,0
2,6/12/19,Cadence Marietta,ALC000560,complaint,-,0,0,0,0,0,0
3,6/12/19,Cadence Marietta,ALC000560,complaint,-,0,0,0,0,0,0
4,2/5/19,Cadence Marietta,ALC000560,Routine / Complaint,-,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
92,1/25/16,Cedar Hill Senior Living Community,PCH008895,complaint,1,0,1,0,0,0,0
93,3/26/15,Cedar Hill Senior Living Community,PCH008895,complaint,1,0,1,0,0,0,0
94,3/6/15,Cedar Hill Senior Living Community,PCH008895,complaint,-,0,0,0,0,0,0
95,2/2/15,Cedar Hill Senior Living Community,PCH008895,Routine / Complaint,1,0,1,0,0,0,0


In [267]:
first_10_facilities_data = pd.DataFrame(facilities_data_chunk1)
first_10_facilities_data

,Name,Address,Type,License,Beds,Website,Avg. Violations
0,Addington Place at the Mills,"550 Barrington Dr, Roswell, GA 30075",Personal Care Home,PCH008668,75,https://www.cedarhurstliving.com/location/addi...,8.4
1,Addington Place of Alpharetta,"762 N Main St, Alpharetta, GA 30009",Assisted Living Community,ALC000209,105,https://www.cedarhurstliving.com/location/addi...,5.6
2,Addington Place of Brunswick,"890 Scranton Rd, Brunswick, GA 31525",Assisted Living Community,ALC000223,65,https://www.cedarhurstliving.com/location/addi...,2.0
3,Addington Place of Dublin,"212 Fairview Park Dr, Dublin, GA 31021",Assisted Living Community,ALC000214,72,https://www.cedarhurstliving.com/location/addi...,0.8
4,Addington Place of Johns Creek,"5050 Kimball Bridge Rd, Alpharetta, GA 30005",Assisted Living Community,ALC000279,72,https://www.cedarhurstliving.com/location/addi...,2.4
5,Alto Senior Living Alpharetta,"4125 North Point Pkwy, Alpharetta, GA 30022",Assisted Living Community,ALC000264,83,https://www.allegroliving.com/alto-alpharetta-...,3.6
6,Alto Senior Living Buckhead,"4804 Roswell Rd, Sandy Springs, GA 30342",Assisted Living Community,ALC000265,83,https://www.allegroliving.com/alto-buckhead-ge...,15.5
7,Alto Senior Living Marietta,"840 Lecroy Dr, Marietta, GA 30068",Assisted Living Community,ALC000266,62,https://www.allegroliving.com/alto-marietta-ge...,1.8
8,Antebellum Arlington Place,"684 Arlington Pl, Macon, GA 31201",Personal Care Home,PCH009095,120,https://www.ssmgrp.com/communities/georgia/ant...,8.0
9,Antebellum Grove Senior Living,"1010 Kathryn Ryals Rd, Warner Robins, GA 31088",Personal Care Home,76030051,69,https://www.ssmgrp.com/communities/georgia/ant...,1.3


In [265]:
first_10_facilities_inspections = pd.DataFrame(inspections_data_chunk1)
first_10_facilities_inspections

,date,name,license,type,Num of Volations,High severe,Medium severe,Harm,Prev1,Prev2,Prev3
0,12/9/19,Addington Place at the Mills,PCH008668,routine,-,0,0,0,0,0,0
1,6/20/19,Addington Place at the Mills,PCH008668,complaint,1,1,0,1,0,0,0
2,6/3/19,Addington Place at the Mills,PCH008668,complaint,2,1,1,1,0,0,0
3,4/12/19,Addington Place at the Mills,PCH008668,Routine / Complaint,6,2,3,0,1,0,0
4,1/12/18,Addington Place at the Mills,PCH008668,complaint,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
97,2/14/17,Antebellum Grove Senior Living,76030051,complaint,-,0,0,0,0,0,0
98,1/11/17,Antebellum Grove Senior Living,76030051,routine,3,0,0,0,0,0,0
99,5/20/16,Antebellum Grove Senior Living,76030051,complaint,-,0,0,0,0,0,0
100,4/18/16,Antebellum Grove Senior Living,76030051,complaint,-,0,0,0,0,0,0


## Combine data list chunks and convert to Data Frame

In [307]:
pd.DataFrame([*facilities_data_chunk1, *facilities_data_chunk2])

,Name,Address,Type,License,Beds,Website,Avg. Violations
0,Addington Place at the Mills,"550 Barrington Dr, Roswell, GA 30075",Personal Care Home,PCH008668,75,https://www.cedarhurstliving.com/location/addi...,8.4
1,Addington Place of Alpharetta,"762 N Main St, Alpharetta, GA 30009",Assisted Living Community,ALC000209,105,https://www.cedarhurstliving.com/location/addi...,5.6
2,Addington Place of Brunswick,"890 Scranton Rd, Brunswick, GA 31525",Assisted Living Community,ALC000223,65,https://www.cedarhurstliving.com/location/addi...,2.0
3,Addington Place of Dublin,"212 Fairview Park Dr, Dublin, GA 31021",Assisted Living Community,ALC000214,72,https://www.cedarhurstliving.com/location/addi...,0.8
4,Addington Place of Johns Creek,"5050 Kimball Bridge Rd, Alpharetta, GA 30005",Assisted Living Community,ALC000279,72,https://www.cedarhurstliving.com/location/addi...,2.4
5,Alto Senior Living Alpharetta,"4125 North Point Pkwy, Alpharetta, GA 30022",Assisted Living Community,ALC000264,83,https://www.allegroliving.com/alto-alpharetta-...,3.6
6,Alto Senior Living Buckhead,"4804 Roswell Rd, Sandy Springs, GA 30342",Assisted Living Community,ALC000265,83,https://www.allegroliving.com/alto-buckhead-ge...,15.5
7,Alto Senior Living Marietta,"840 Lecroy Dr, Marietta, GA 30068",Assisted Living Community,ALC000266,62,https://www.allegroliving.com/alto-marietta-ge...,1.8
8,Antebellum Arlington Place,"684 Arlington Pl, Macon, GA 31201",Personal Care Home,PCH009095,120,https://www.ssmgrp.com/communities/georgia/ant...,8.0
9,Antebellum Grove Senior Living,"1010 Kathryn Ryals Rd, Warner Robins, GA 31088",Personal Care Home,76030051,69,https://www.ssmgrp.com/communities/georgia/ant...,1.3


In [317]:
facilities_data_all = []

for i in range(1,41):
    facilities_data_all = [*facilities_data_all, *globals()['facilities_data_chunk%s' % (i)]]

facilities_data_all_df = pd.DataFrame(facilities_data_all)
facilities_data_all_df

,Name,Address,Type,License,Beds,Website,Avg. Violations
0,Addington Place at the Mills,"550 Barrington Dr, Roswell, GA 30075",Personal Care Home,PCH008668,75,https://www.cedarhurstliving.com/location/addi...,8.4
1,Addington Place of Alpharetta,"762 N Main St, Alpharetta, GA 30009",Assisted Living Community,ALC000209,105,https://www.cedarhurstliving.com/location/addi...,5.6
2,Addington Place of Brunswick,"890 Scranton Rd, Brunswick, GA 31525",Assisted Living Community,ALC000223,65,https://www.cedarhurstliving.com/location/addi...,2.0
3,Addington Place of Dublin,"212 Fairview Park Dr, Dublin, GA 31021",Assisted Living Community,ALC000214,72,https://www.cedarhurstliving.com/location/addi...,0.8
4,Addington Place of Johns Creek,"5050 Kimball Bridge Rd, Alpharetta, GA 30005",Assisted Living Community,ALC000279,72,https://www.cedarhurstliving.com/location/addi...,2.4
...,...,...,...,...,...,...,...
393,Winnwood Retirement Community,"100 Whitlock Ave NW, Marietta, GA 30064",Assisted Living Community,ALC000085,70,https://winnwoodretire.com/lifestyles/independ...,0.5
394,Winthrop Court,"10 US-411, Rome, GA 30161",Personal Care Home,57030021,100,https://www.winthropseniorliving.com/,4.5
395,Woodland Ridge Senior Living,"4005 S Cobb Dr, Smyrna, GA 30080",Assisted Living Community,ALC000008,96,tel:770-431-7055,4.0
396,"Woods Senior Living Community, The","1401 Macon Rd, Griffin, GA 30224",Personal Care Home,126030011,40,tel:770-412-6005,3.5


In [318]:
inspections_data_all = []

for i in range(1,41):
    inspections_data_all = [*inspections_data_all, *globals()['inspections_data_chunk%s' % (i)]]

inspections_data_all_df = pd.DataFrame(inspections_data_all)
inspections_data_all_df

,date,name,license,type,Num of Volations,High severe,Medium severe,Harm,Prev1,Prev2,Prev3
0,12/9/19,Addington Place at the Mills,PCH008668,routine,-,0,0,0,0,0,0
1,6/20/19,Addington Place at the Mills,PCH008668,complaint,1,1,0,1,0,0,0
2,6/3/19,Addington Place at the Mills,PCH008668,complaint,2,1,1,1,0,0,0
3,4/12/19,Addington Place at the Mills,PCH008668,Routine / Complaint,6,2,3,0,1,0,0
4,1/12/18,Addington Place at the Mills,PCH008668,complaint,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
3270,11/19/18,Yellow Brick House,44030041,other,-,0,0,0,0,0,0
3271,8/21/18,Yellow Brick House,44030041,other,-,0,0,0,0,0,0
3272,5/15/18,Yellow Brick House,44030041,complaint,11,1,0,1,0,0,0
3273,12/14/16,Yellow Brick House,44030041,routine,11,0,6,0,1,0,0


In [321]:
# Save resulting data into csv files:
facilities_data_all_df.to_csv("Facilities_data.csv", index=False)
pd.read_csv("Facilities_data.csv").head()

,Name,Address,Type,License,Beds,Website,Avg. Violations
0,Addington Place at the Mills,"550 Barrington Dr, Roswell, GA 30075",Personal Care Home,PCH008668,75,https://www.cedarhurstliving.com/location/addi...,8.4
1,Addington Place of Alpharetta,"762 N Main St, Alpharetta, GA 30009",Assisted Living Community,ALC000209,105,https://www.cedarhurstliving.com/location/addi...,5.6
2,Addington Place of Brunswick,"890 Scranton Rd, Brunswick, GA 31525",Assisted Living Community,ALC000223,65,https://www.cedarhurstliving.com/location/addi...,2.0
3,Addington Place of Dublin,"212 Fairview Park Dr, Dublin, GA 31021",Assisted Living Community,ALC000214,72,https://www.cedarhurstliving.com/location/addi...,0.8
4,Addington Place of Johns Creek,"5050 Kimball Bridge Rd, Alpharetta, GA 30005",Assisted Living Community,ALC000279,72,https://www.cedarhurstliving.com/location/addi...,2.4


In [322]:
# Save resulting inspections data into csv files:
inspections_data_all_df.to_csv("Inspections_data.csv", index=False)
pd.read_csv("Inspections_data.csv").head()

,date,name,license,type,Num of Volations,High severe,Medium severe,Harm,Prev1,Prev2,Prev3
0,12/9/19,Addington Place at the Mills,PCH008668,routine,-,0,0,0,0,0,0
1,6/20/19,Addington Place at the Mills,PCH008668,complaint,1,1,0,1,0,0,0
2,6/3/19,Addington Place at the Mills,PCH008668,complaint,2,1,1,1,0,0,0
3,4/12/19,Addington Place at the Mills,PCH008668,Routine / Complaint,6,2,3,0,1,0,0
4,1/12/18,Addington Place at the Mills,PCH008668,complaint,1,0,1,0,0,0,0
